# Uni-MuMER Project - Kaggle Setup

Notebook này cài đặt và chạy Uni-MuMER Project trên Kaggle với Python 3.10 và conda

In [1]:
# Cài đặt Miniconda vào thư mục có quyền ghi
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /kaggle/working/miniconda
!rm Miniconda3-latest-Linux-x86_64.sh

# Đồng ý điều khoản của Anaconda (Bắt buộc để cài đặt các package từ channel main/r)
!/kaggle/working/miniconda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!/kaggle/working/miniconda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Tạo môi trường unimumer với Python 3.10
!/kaggle/working/miniconda/bin/conda create -n unimumer python=3.10 -y

--2025-12-27 05:41:35--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156772981 (150M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 149.51M   185MB/s    in 0.8s    

2025-12-27 05:41:36 (185 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [156772981/156772981]

PREFIX=/kaggle/working/miniconda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best 

In [23]:

# --- BƯỚC 2: CLONE & TẢI MODEL ---
ACTIVATE = "source /kaggle/working/miniconda/bin/activate unimumer"

!git clone https://github.com/KhaiHASO/Uni-MuMER-FineTune.git
%cd Uni-MuMER-FineTune

# Đã vào trong Uni-MuMER-FineTune, giờ mới có folder Uni-MuMER-Qwen2.5-VL-3B
MODEL_DIR = "Uni-MuMER-Qwen2.5-VL-3B"

# Tải model (Thêm -L để xử lý redirect từ Hugging Face)
!{ACTIVATE} && wget -L -O {MODEL_DIR}/model-00001-of-00002.safetensors "https://huggingface.co/phxember/Uni-MuMER-Qwen2.5-VL-3B/resolve/main/model-00001-of-00002.safetensors?download=true"
!{ACTIVATE} && wget -L -O {MODEL_DIR}/model-00002-of-00002.safetensors "https://huggingface.co/phxember/Uni-MuMER-Qwen2.5-VL-3B/resolve/main/model-00002-of-00002.safetensors?download=true"



Cloning into 'Uni-MuMER-FineTune'...
remote: Enumerating objects: 652, done.
remote: Counting objects: 100% (652/652), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 652 (delta 221), reused 641 (delta 210), pack-reused 0 (from 0)
Receiving objects: 100% (652/652), 14.40 MiB | 23.11 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/kaggle/working/Uni-MuMER-FineTune
--2025-12-27 06:00:48--  https://huggingface.co/phxember/Uni-MuMER-Qwen2.5-VL-3B/resolve/main/model-00001-of-00002.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 13.226.251.20, 13.226.251.66, 13.226.251.81, ...
Connecting to huggingface.co (huggingface.co)|13.226.251.20|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/68353d4f2da10c54bbbd6a4c/e3c6bba6468762f3f5828469c76764a6152f96fc6aa9f102ef8b917f2aafc7c7?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=cas%2F2

In [ ]:
# --- BƯỚC 3: CÀI ĐẶT THƯ VIỆN ---
# Lưu ý: CUDA 12.1 hoặc 12.4 ổn định hơn trên Kaggle T4, bạn có thể thử 12.8 nếu cần
#!{ACTIVATE} && conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia -y
!{ACTIVATE} && pip install -r requirements.txt

# Cài đặt LLaMA-Factory
%cd train/LLaMA-Factory
!{ACTIVATE} && pip install -e .

# Quay lại thư mục gốc dự án
%cd ../..

In [ ]:
!source /kaggle/working/miniconda/bin/activate unimumer && \
 MPLBACKEND=Agg llamafactory-cli train train/Uni-MuMER-train.yaml


[INFO|2025-12-27 06:03:59] llamafactory.launcher:143 >> Initializing 2 distributed tasks at: 127.0.0.1:49795
W1227 06:04:00.669000 693 site-packages/torch/distributed/run.py:792] 
W1227 06:04:00.669000 693 site-packages/torch/distributed/run.py:792] *****************************************
W1227 06:04:00.669000 693 site-packages/torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1227 06:04:00.669000 693 site-packages/torch/distributed/run.py:792] *****************************************
[WARNING|2025-12-27 06:04:07] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-12-27 06:04:07] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
[INFO|2025-12-27 06:04:07] llamafactory.hparam